In [1]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost
from agent import Agent
from creator import Creator
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime
from autogen_core import AgentId
from messages import Message
import glob
import importlib
import os

In [2]:

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

In [3]:
creator_worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
name = "Creator"
await creator_worker.start()
result = await Creator.register(creator_worker, name, lambda: Creator(name))
print(result)
creator_id = AgentId(name, "default")

AgentType(type='Creator')


In [4]:
async def create_agents():
    how_many_agents = 2
    new_agents = 0
    count = 1

    while new_agents < how_many_agents:
        name = f"agent{count}.py"
        if not os.path.exists(name):
            print(f"Creating agent {name}")
            await creator_worker.send_message(Message(content=name), creator_id)
            new_agents += 1
        count += 1

await create_agents()

About to generate agent: agent3.py
INFO:autogen_core.trace:About to generate agent: agent3.py
Generated agent: agent3.py
INFO:autogen_core.trace:Generated agent: agent3.py
Wrote agent to file: agent3.py
INFO:autogen_core.trace:Wrote agent to file: agent3.py
About to generate agent: agent4.py
INFO:autogen_core.trace:About to generate agent: agent4.py
Generated agent: agent4.py
INFO:autogen_core.trace:Generated agent: agent4.py
Wrote agent to file: agent4.py
INFO:autogen_core.trace:Wrote agent to file: agent4.py


Creating agent agent3.py
Creating agent agent4.py


In [5]:
workers = []
agent_ids = []
count = 1

async def run_agents():

    agent_files = glob.glob("agent*.py")
    module_names = [os.path.splitext(file)[0] for file in agent_files]
        
    for name in module_names:
        try:
            module = importlib.import_module(name)
            worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
            await worker.start()
            await module.Agent.register(worker, name, lambda: module.Agent(name))
            workers.append(worker)
            agent_id = AgentId(name, "default")
            agent_ids.append(agent_id)
            response = await worker.send_message(Message(content="Please give me brief business idea in 2-3 sentences."), agent_id)
            print(response.content)
        except Exception as e:
            print(f"Error with this agent: {name}: {e}")

await run_agents()

Imagine a platform that connects businesses with freelance accessibility consultants who specialize in digital inclusivity. This service would offer tailored audits and recommendations for websites and applications, ensuring compliance with accessibility standards while enhancing user experience for individuals with disabilities. By fostering a culture of inclusivity, this platform not only empowers users but also helps businesses tap into a wider audience, driving both social impact and economic growth.
Consider launching a subscription service that delivers curated art and music experiences directly to consumers' homes. Each month, subscribers receive a box containing limited-edition prints from emerging artists, a playlist of new music curated by genre or theme, and an accompanying booklet that explores the cultural significance of the featured works. This idea not only supports artists and musicians but also fosters a deeper appreciation of art and music history among subscribers, 

In [ ]:
for worker in workers + [creator_worker]:
    try:
        await worker.stop()
    except Exception as e:
        print(f"Error stopping worker: {e}")
try:
    await host.stop()   
except Exception as e:
    print(f"Error stopping host: {e}")